In [ ]:
#subdir in results
#result_path = 'results/simcyp298/'
result_path = 'results/pmc_sim_ami_128/'


# load puptator file
#filelist = ['data/Tafamidis97_filtered.pubtator']
#filelist = ['data/SimCyp298_filtered.pubtator']

# pmc
filelist = ['data/pmc_simvastatin_amiodarone_filtered_128.pubtator']
#filelist = ['data/pmc_simvastatin_erythromycin_filtered_108.pubtator']
#filelist = ['data/pmc_sim_cyp_100_filtered.pubtator']

In [ ]:
import os
print('remove snorkel db...')
os.remove("snorkel.db") 
print('snorkel db removed')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel import SnorkelSession
from snorkel.parser import CorpusParser, Spacy, StanfordCoreNLPServer
from pubtator import PubTatorDocPreprocessor, PubTatorTagProcessor, PubTatorParser
from time import time

parser = "spacy"
num_procs = 1

start_ts = time()


for fp in filelist:
    doc_preprocessor = PubTatorDocPreprocessor(fp, annotations=False, debug=True)
    #arser = Spacy() if parser == "spacy" else StanfordCoreNLPServer()
    parser = PubTatorParser(stop_on_err=False)
    corpus_parser = CorpusParser(parser=parser)
    corpus_parser.apply(doc_preprocessor, parallelism=num_procs, clear=True)
    end_ts = time()

print("\nDONE in {}".format((time() - start_ts)))

In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

In [ ]:
import os


try:  
    os.mkdir(result_path)
except OSError:  
    print ("Creation of the directory {} failed because it may exists".format(result_path))
else:  
    print ("Successfully created the directory %s " % result_path)


In [ ]:
from snorkel.models import Document

all_docs = session.query(Document).all()

print('Amount of docs: {}'.format(len(all_docs)))
with open(result_path + 'doc_mapping.tsv', 'w') as f:
    f.write('{}\t{}'.format('snorkel_id', 'pmid'))
    for doc in all_docs:
        f.write('\n{}\t{}'.format(doc.id, doc.name))
print('Finished')

In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print("Loading all sentences from db...")
all_sents = session.query(Sentence).all()
print("Loading complete!")


print('Amount of sentences: {}'.format(len(all_sents)))

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalDisease, ['Chemical', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalDisease).filter(ChemicalDisease.split == k).count())
    

from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_disease.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).order_by(ChemicalDisease.id).all()
print("{} candidates load from db!".format(len(all_cands)))


print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id', 'chemical_cid', 'chemical_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_disease_association.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'disease_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalGeneInteraction = candidate_subclass('ChemicalGeneInteraction', ['chemical', 'gene'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalGeneInteraction, ['Chemical', 'Gene'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_interaction.lstm')


print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == 0).order_by(ChemicalGeneInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_interaction.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

GeneDiseaseInteraction = candidate_subclass('GeneDiseaseInteraction', ['gene', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(GeneDiseaseInteraction, ['Gene', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_disease_interaction.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == 0).order_by(GeneDiseaseInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_disease_interaction.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'disease_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor


GeneChemicalMetabolism = candidate_subclass('GeneChemicalMetabolism', ['gene', 'chemical'])
candidate_gene_chemical_metabolism_extractor = PretaggedCandidateExtractor(GeneChemicalMetabolism, ['Gene', 'Chemical'])

for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_metabolism_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == k).count())
    
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_chemical_metabolism.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == 0).order_by(GeneChemicalMetabolism.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'chemical_cid', 'chemical_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_chemical_metabolism.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'chemical_cid')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [ ]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor



ChemicalGeneInhibition = candidate_subclass('ChemicalGeneInhibition', ['chemical', 'gene'])
candidate_gene_chemical_inhibit_extractor = PretaggedCandidateExtractor(ChemicalGeneInhibition, ['Chemical', 'Gene'])


for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_inhibit_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_inhibition.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == 0).order_by(ChemicalGeneInhibition.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_inhibition.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')